In [9]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import  sys
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [29]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
from src.utils import get_all_channels_messages, preprocess_text


In [30]:
def build_model(corpus, word_to_id):
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                        id2word=word_to_id,
                                        num_topics=5,
                                        random_state=100,
                                        update_every=1,
                                        chunksize=100,
                                        passes=10,
                                        alpha='auto',
                                        per_word_topics=True)    
    return lda_model

def show_topics(lda_model):
    pprint(lda_model.show_topics(formatted=False))

def get_top_topics(df):
    df, word_list, word_to_id, corpus = prepare_data(df)
    lda_model = build_model(corpus, word_to_id)

    # Show the top 10 topics
    show_topics(lda_model)
    
    # Visualize the top 10 topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, word_to_id)
    return LDAvis_prepared

def prepare_data(df):
    print(df)
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    sentence_list = [tweet for tweet in df['cleaned_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word
    word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    corpus = [word_to_id.doc2bow(tweet) for tweet in word_list]
    
    return df, word_list, word_to_id, corpus


In [31]:
slack_data_path = os.path.abspath('../anonymized')
all_channels = SlackDataLoader(slack_data_path).get_channels()

# Top 10 topics from all channels

In [32]:
df = get_all_channels_messages(all_channels)

get_top_topics(df)

[('morning boss', '1666940798.397899'), ("*hot seat announcement! <!here>*\n_*good morning all,*_\nthe long-awaited day/time is here……guess whom we have in...\nthis is your only chance to ask all the questions you've been curious about without getting in trouble:rolling_on_the_floor_laughing:…use it wisely and let's make the seat very *hot! i repeat very hot like a hito, like a bomb*:boom:\nwe are *more than* happy to announce today's hotseat guest.\nwe are very proud to host  our very own  <@u03uun8m4rx>", '1666941319.221839'), ("you can find the updated list of the hot seater's quake books <https://www.notion.so/quake-books-1a622f5c9a1f4a418126fcc4ce05aa01|here>. have fun reading.", '1666948195.284989'), ('*community building session reminder!*:timer_clock: *<!here>*\n*please note that cbs is on in the next Joannemin*\n• *session: my movie life*\n• *time*: from 12:00 pm - 12:30 pm utc\n• platform: slack\n', '1666957800.615229'), ('hello', '1666958458.036999'), ('selam', '1666958469.3

TypeError: list indices must be integers or slices, not str